In [26]:
from msedge.selenium_tools import Edge, EdgeOptions
from selenium.webdriver.common.by import By
from selenium.webdriver.support.wait import WebDriverWait
from selenium.webdriver.support import expected_conditions
from selenium.common.exceptions import NoSuchElementException
import os

In [30]:
def wait_until(browser, xpath):
    WebDriverWait(browser, 25).until(
        expected_conditions.presence_of_element_located((By.XPATH, xpath)))

In [32]:
def scrap_memes(number_of_memes):
    options = EdgeOptions()
    options.use_chromium = True
    # options.add_argument('--headless')
    options.add_argument('--disable-notifications')

    options.add_experimental_option("prefs", {
        "download.default_directory":  os.getcwd() + '\memes',
        "download.prompt_for_download": "false",
        "download.directory_upgrade": "true",
        "safebrowsing.enabled": "true"})

    browser = Edge('msedgedriver.exe', options=options)
    browser.get('https://web.whatsapp.com/')
    wait_until(browser, '//span[@title="Free memes"]')
    browser.find_element(By.XPATH, '//span[@title="Free memes"]').click()
    wait_until(browser, '//*[@id="main"]/header/div[2]/div[1]/div/span')
    browser.find_element(By.XPATH, '//*[@id="main"]/header/div[2]/div[1]/div/span').click()
    wait_until(browser, "//span[text()='Archivos, enlaces y documentos']")
    browser.find_element(By.XPATH, "//span[text()='Archivos, enlaces y documentos']").click()
    wait_until(browser, '//*[@class="_23fpc"]')
    browser.find_element(By.XPATH, '//*[@class="_23fpc"]').click()
    for _ in range(number_of_memes):
        try:
            wait_until(browser, '//span[@data-icon="download"]')
            browser.find_element(By.XPATH, '//video')
            browser.find_element(By.XPATH, '//span[@data-icon="chevron-left"]').click()
        except NoSuchElementException:
            browser.find_element(By.XPATH, '//span[@data-icon="download"]').click()
            browser.find_element(By.XPATH, '//span[@data-icon="chevron-left"]').click()